<a href="https://colab.research.google.com/github/am610/DeepFake/blob/main/Sentinel_II_Video_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from tensorflow.keras.models import load_model
import imageio
from IPython.display import HTML
from scipy.stats import ttest_1samp
import dlib
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import matplotlib.animation as animation
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import warnings
warnings.filterwarnings("ignore")

In [4]:
def display(driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = []
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

In [5]:
# Hypothesis 1 sample t-test. 
# Assuming Null hypothesis  = Real Video. 
# Alternate Hypothesis = Fake Video
def hypothesis(hypo):
  tset, pval = ttest_1samp(hypo, 1)
  if (pval<0.05):
    print('Fake Video')
  else:
    print('Real Video') 

In [6]:
# Based on the detection of individual frames per video. If they are flagged as real or fake. 
def result(a,b):
  r = a/b
  if r<0.7:
    print('Fake Video')
  else:
    print('Real Video')  

In [7]:
def video_test(path):
    warnings.filterwarnings("ignore")
    pr_data = []
    hypo=[]
    count=0;total=0;
    detector = dlib.get_frontal_face_detector()
    cap = cv2.VideoCapture(path)
    frameRate = cap.get(5)
    while cap.isOpened():
        frameId = cap.get(1)
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % ((int(frameRate)+1)*1) == 0:
            face_rects, scores, idx = detector.run(frame, 0)
            for i, pos in enumerate(face_rects):
                left = pos.left()
                top = pos.top()
                right = pos.right()
                bottom = pos.bottom()
                total+=1
                crop_img = frame[top:bottom, left:right]
                if (np.shape(crop_img)[0]==0):
                  continue
                elif (np.shape(crop_img)[1]==0): 
                  continue
                elif (np.shape(crop_img)[2]==0): 
                  continue   
                data = img_to_array(cv2.resize(crop_img, (128, 128))).flatten() / 255.0
                #print(np.shape(data))
                data = data.reshape(-1, 128, 128, 3)
                #print(model2.predict_classes(data))
                hypo.append(model2.predict_classes(data)[0])
                if(model2.predict_classes(data)==1):
                  count+=1

    result(count,total)
    #hypothesis(hypo)
    #return (count/total)

# Add this folder to your google drive https://drive.google.com/drive/folders/1Zppu1TunpSD46r9n0rQhdrMVBrqLcIUx?usp=sharing



In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
DATA_PREFIX = '/content/gdrive/My Drive/Videos_DF/v/v2'

### Models : We have two trained models (in next cell) any one can be used. Generated from the other Notebook. 

In [10]:
#model = load_model(DATA_PREFIX+'/dataset/model.h5')
model2 = load_model(DATA_PREFIX+'/dataset/model-2.h5')

# Test Video : Choose any one video from below, by uncommenting it. 

##### User can provide links of other videos present in the folder or other videos, by providing proper links.

In [11]:
# REAL VIDEO INPUT TEST CASES

#path = DATA_PREFIX+'/real/23408559174326383095810043557.mp4' 
#path = DATA_PREFIX+'/real/951434276370648925832295649036.mp4'
#path = DATA_PREFIX+'/real/990528374445753215169375502905.mp4'
#path = DATA_PREFIX+'/real/989873876027077232709160757936.mp4'
#path = DATA_PREFIX+'/real/983964079718160933185214836121.mp4'
#path = DATA_PREFIX+'/real/986829114623101283200591746149.mp4'
#---------------------------------------------------------------

# FAKE VIDEO INPUT TEST CASES
#path = DATA_PREFIX+'/fake/914588625698788367603311258375.mp4' 
#path = DATA_PREFIX+'/fake/904237124363242243229103239952.mp4' 
path = DATA_PREFIX+'/fake/982448469770167074727251760045.mp4'

# Check Video Here: Real or Fake

In [13]:

# To see the video. 
# Takes time to load. So you **may skip** it, if you don't want to wait. 


reader = imageio.get_reader(path)
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

HTML(display(driving_video).to_html5_video())


#### Result

In [14]:
video_test(path)

Fake Video
